<a href="https://colab.research.google.com/github/4L3M4R/cerbero/blob/main/cerbero-pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
#             CERBERO PREMARKET
#   Descarga noticias y calcula sentiment
# ===============================================

import os
import pandas as pd
import feedparser
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import datetime
from urllib.parse import quote

# ===============================================
#             CONFIGURACIÓN
# ===============================================

# Lista de activos
activos = {}
with open("activos.txt", "r") as f:
    for line in f:
        symbol, source, search_name = line.strip().split(":")
        activos[symbol.strip()] = {
            "source": source.strip().lower(),
            "search_name": search_name.strip()
        }

# Inicialización Sentiment
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# ===============================================
#           FUNCIONES AUXILIARES
# ===============================================

def registrar_log(message, log_file="run_summary_pre.log"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as log:
        log.write(f"[{timestamp}] {message}\n")

def descargar_noticias_y_calcular_sentiment(symbol, search_name):
    os.makedirs("logs", exist_ok=True)

    query = quote(search_name)
    feed = feedparser.parse(f"https://news.google.com/rss/search?q={query}")
    noticias = []
    for entry in feed.entries:
        noticias.append({
            "timestamp": entry.published,
            "title": entry.title,
            "link": entry.link
        })
    df = pd.DataFrame(noticias)
    if df.empty:
        registrar_log(f"{symbol} - No se encontraron noticias")
        return

    # Calcular sentiment
    df["vader_sentiment"] = df["title"].apply(lambda x: vader_analyzer.polarity_scores(x)["compound"])
    df["finbert_sentiment"] = 0  # Placeholder para FinBERT

    # Promedios
    vader_promedio = df["vader_sentiment"].mean()
    finbert_promedio = df["finbert_sentiment"].mean()

    # Actualizar _datos.txt
    actualizar_sentimiento(symbol, vader_promedio, finbert_promedio)

    # Guardar log de noticias nuevas
    today_str = pd.Timestamp.utcnow().date()
    df.to_csv(f"logs/{symbol}_nuevas_agregadas_{today_str}.csv", index=False)
    registrar_log(f"{symbol} - Guardadas {len(df)} noticias con sentiment")

def actualizar_sentimiento(symbol, vader, finbert=0):
    filename = f"{symbol}_datos.txt"
    hoy = pd.Timestamp.utcnow().date().strftime("%Y-%m-%d")
    hoy_date = pd.to_datetime(hoy).date()

    if not os.path.exists(filename):
        df = pd.DataFrame({
            "timestamp": [hoy_date],
            "vader_sentiment": [vader],
            "finbert_sentiment": [finbert]
        })
        df.to_csv(filename, sep=",", index=False)
        registrar_log(f"{symbol} - Archivo creado con sentiment inicial")
        return

    # Leer archivo existente
    df = pd.read_csv(filename, sep=",")
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce").dt.date

    # Actualizar o agregar línea de hoy
    if hoy_date in df["timestamp"].values:
        df.loc[df["timestamp"] == hoy_date, "vader_sentiment"] = vader
        df.loc[df["timestamp"] == hoy_date, "finbert_sentiment"] = finbert
        registrar_log(f"{symbol} - Sentiment de hoy actualizado: Vader={vader:.4f}, FinBERT={finbert:.4f}")
    else:
        nueva = {"timestamp": hoy_date, "vader_sentiment": vader, "finbert_sentiment": finbert}
        df = pd.concat([df, pd.DataFrame([nueva])], ignore_index=True)
        registrar_log(f"{symbol} - Nueva línea agregada: Vader={vader:.4f}, FinBERT={finbert:.4f}")

    df.to_csv(filename, sep=",", index=False)

# ===============================================
#           EJECUCIÓN PRINCIPAL
# ===============================================

for symbol, info in activos.items():
    descargar_noticias_y_calcular_sentiment(symbol, info["search_name"])

print("Completed premarket")
